In [1]:
import sys
import struct
import argparse

import numpy as np
import openhd as hd

/home/hpc/iwi3/iwi3083h/.local/lib/python3.8/site-packages/skcuda/cublas.py:284: UserWarning: creating CUBLAS context to get version number
  warnings.warn('creating CUBLAS context to get version number')


In [2]:
!rm -rf ~/.openhd

# Iris

In [3]:
from sklearn import datasets

iris = datasets.load_iris()

In [4]:
X = iris.data
y = iris.target


In [5]:
X.shape

(150, 4)

In [6]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
# X.dtype = np.float32

In [7]:
X.shape

(150, 4)

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

In [9]:
X_train.shape

(135, 4)

In [10]:
X_train[0]

array([0.55555556, 0.54166667, 0.62711864, 0.625     ])

In [11]:
N = X_train.shape[0]
F = X_train.shape[1]
N, F

(135, 4)

In [12]:
Q = 100
D = 10000
hd.init(D=D, context=globals())


In [13]:
@hd.run
def create_random_bases():
    id_base = hd.draw_random_hypervector()
    level_base = hd.draw_random_hypervector()
    return id_base, level_base


@hd.run
def create_ids(F, id_base):
    id_hvs = hd.hypermatrix(F) # np.zeros(F, N) (not the empty list) 
    for f in range(F):
        id_hvs[f] = hd.permute(id_base, f)

    return id_hvs

@hd.run
def create_levels(Q, level_base):
    level_hvs = hd.hypermatrix(Q+1) # np.zeros((Q+1), N) (not the empty list)
    for q in range(Q+1):
        idx = int(q/float(Q) * D) / 2
        level_hvs[q] = hd.flip(level_base, idx)
        level_hvs[q] = hd.shuffle(level_hvs[q], 0)

    return level_hvs

@hd.run
def assoc_search(class_hvs, hv_matrix_tst):
    ret = hd.search(class_hvs, hv_matrix_tst)
    return ret

def validate(labels, pred_labels):
    n_correct = (pred_labels == labels).sum()
    n_labels = len(labels)
    print(n_correct, n_labels, n_correct / float(n_labels) * 100)


In [14]:
with hd.utils.timing("Base hypervectors"):
    id_base, level_base = create_random_bases()
    id_hvs = create_ids(F, id_base)
    level_hvs = create_levels(Q, level_base)


[ERROR]	jit.date_type_mutator	
<class '_ast.Module'> : 
  <class '_ast.Assign'> :  [__ARG__id_base : __one__]
  <class '_ast.Assign'> :  [__ARG__level_base : __one__]

{'id_base': 'hypervec_type', 'level_base': 'hypervec_type', '__ARG__id_base': 'float*', '__ARG__level_base': 'float*', 'id_base_22b19319': <class 'float'>, 'level_base_5541502b': <class 'float'>}
[ERROR]	jit.date_type_mutator	
<class '_ast.Module'> : 
  <class '_ast.For'> :  <RPT: F>
    <class '_ast.Assign'> :  [__ARG__id_hvs : __ARG__id_base]

{'f': <class 'int'>, 'F': <class 'int'>, 'id_hvs': 'hypermat_type', 'id_base': 'hypervec_type', '__ARG__id_base': 'float*', '__ARG__id_hvs': 'float*', 'id_base_f_16a56737': <class 'float'>, 'id_hvs_f_00177a23': <class 'float'>}
[ERROR]	jit.date_type_mutator	
<class '_ast.Module'> : 
  <class '_ast.Assign'> :  [level_hvs_q_7acb546c : __ARG__level_base]
  <class '_ast.Assign'> :  [__ARG__level_hvs : level_hvs_q_7acb546c]

{'idx': <class 'int'>, 'q': 'int', 'Q': <class 'int'>, 'D': 

In [15]:
def preprocesser(
        org_feature, cnv_feature, # Predefined argument (single feature)
        Q, level_hvs, id_hvs): # arguments passed by args
    cnv_feature = int(org_feature * Q)


def encoder(
        input_features, output_hypervector, # Predefined arguments
        Q, level_hvs, id_hvs): # arguments passed by args
    for f in range(F):
        output_hypervector += level_hvs[input_features[f]] * id_hvs[f]

In [16]:
with hd.utils.timing("Encode training"):
    hv_matrix = hd.encode(
            encoder, extra_args = (Q, level_hvs, id_hvs),
            feature_matrix = X_train,
#             preprocess_function = preprocesser # optional
            )

[ERROR]	jit.date_type_mutator	
<class '_ast.Module'> : 
  <class '_ast.Assign'> :  [output_hypervector___n_____base_n___2964e176 : __ARG__output_hypervector]
  <class '_ast.For'> :  <RPT: F>
    <class '_ast.Assign'> :  [output_hypervector___n_____base_n___2964e176 : __ARG__id_hvs*__ARG__level_hvs + output_hypervector___n_____base_n___2964e176]
  <class '_ast.Assign'> :  [__ARG__output_hypervector : output_hypervector___n_____base_n___2964e176]

{'__n__': <class 'int'>, '__blockIdx_y__': <class 'int'>, '__base_n__': 'int', '__N__': <class 'int'>, '__blockDim_x__': <class 'int'>, '__F__': <class 'int'>, '__threadIdx_x__': <class 'int'>, 'F_PER_THREAD': <class 'int'>, 'sample_idx_in_stream': <class 'int'>, '__stream__': 'int', '__M__': <class 'int'>, '__f__': <class 'int'>, '__f_idx__': <class 'int'>, 'original_feature': <class 'float'>, 'input_features': 'np_float_array_type', 'preprocessed_feature': <class 'float'>, '__shared_features__': 'np_float_array_type', '__d__': <class 'int'>, 

In [17]:
with hd.utils.timing("Encode testing"):
    hv_matrix_test = hd.encode(
            encoder, extra_args = (Q, level_hvs, id_hvs),
            feature_matrix = X_test,
#             preprocess_function = preprocesser # optional
            )

[ERROR]	jit.date_type_mutator	
<class '_ast.Module'> : 
  <class '_ast.Assign'> :  [output_hypervector___n_____base_n___2964e176 : __ARG__output_hypervector]
  <class '_ast.For'> :  <RPT: F>
    <class '_ast.Assign'> :  [output_hypervector___n_____base_n___2964e176 : __ARG__id_hvs*__ARG__level_hvs + output_hypervector___n_____base_n___2964e176]
  <class '_ast.Assign'> :  [__ARG__output_hypervector : output_hypervector___n_____base_n___2964e176]

{'__n__': <class 'int'>, '__blockIdx_y__': <class 'int'>, '__base_n__': 'int', '__N__': <class 'int'>, '__blockDim_x__': <class 'int'>, '__F__': <class 'int'>, '__threadIdx_x__': <class 'int'>, 'F_PER_THREAD': <class 'int'>, 'sample_idx_in_stream': <class 'int'>, '__stream__': 'int', '__M__': <class 'int'>, '__f__': <class 'int'>, '__f_idx__': <class 'int'>, 'original_feature': <class 'float'>, 'input_features': 'np_float_array_type', 'preprocessed_feature': <class 'float'>, '__shared_features__': 'np_float_array_type', '__d__': <class 'int'>, 

In [18]:
n_classes = len(set(y))
n_classes

3

In [19]:
@hd.run
def train(class_hvs, hv_matrix, labels, N):
    for idx in range(N): # Iterate through each image
        class_hvs[labels[idx]] += hv_matrix[idx]
    return class_hvs

In [20]:
class_hvs = hd.hypermatrix(n_classes)

In [21]:
class_hvs.to_numpy().shape

(3, 10000)

In [22]:
y_train

array([1, 2, 1, 1, 2, 0, 2, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 1,
       0, 0, 0, 2, 1, 1, 0, 0, 1, 2, 2, 1, 2, 1, 2, 1, 0, 2, 1, 0, 0, 0,
       1, 2, 0, 0, 0, 1, 0, 1, 2, 0, 1, 2, 0, 2, 2, 1, 1, 2, 1, 0, 1, 2,
       0, 0, 1, 1, 0, 2, 0, 0, 1, 1, 2, 1, 2, 2, 1, 0, 0, 2, 2, 0, 0, 0,
       1, 2, 0, 2, 2, 0, 1, 1, 2, 1, 2, 0, 2, 1, 2, 1, 1, 1, 0, 1, 1, 0,
       1, 2, 2, 0, 1, 2, 2, 0, 2, 0, 1, 2, 2, 1, 2, 1, 1, 2, 2, 0, 1, 2,
       0, 1, 2])

In [23]:
hv_matrix.to_numpy().shape

(135, 10000)

In [24]:
y_train = np.array(y_train, dtype = np.int32)
y_test = np.array(y_test, dtype = np.int32)

# One Shot Learning

In [25]:
@hd.run
def one_shot_train(class_hvs, hv_matrix, labels, N):
    for idx in range(N): # Iterate through each image
        class_hvs[labels[idx]] += hv_matrix[idx]
    return class_hvs

In [26]:

class_hvs = hd.hypermatrix(n_classes)

class_hvs = one_shot_train(class_hvs, hv_matrix, y_train, N)

validate(y_test, hd.search(class_hvs, hv_matrix_test).to_numpy())


[ERROR]	jit.date_type_mutator	
<class '_ast.Module'> : 
  <class '_ast.For'> :  <RPT: N>
    <class '_ast.Assign'> :  [__ARG__class_hvs : __ARG__class_hvs + __ARG__hv_matrix]

{'idx': <class 'int'>, 'N': <class 'int'>, 'class_hvs': 'hypermat_type', 'labels': 'np_int_array_type', 'hv_matrix': 'hypermat_type', '__ARG__hv_matrix': 'float*', '__ARG__labels': 'int*', '__ARG__labels__STRIDE__': 'const int', '__ARG__class_hvs': 'float*', 'hv_matrix_idx_ead026f6': <class 'float'>, 'labels_idx_b9c77ce4': <class 'int'>, 'class_hvs_idx_labels_df62a0b1': <class 'float'>}
2 15 13.333333333333334


In [27]:
@hd.run
def single_pass(hv_matrix, labels, N, n_classes):
    class_hvss = hd.hypermatrix(n_classes)

    for idx in range(N):
        class_hvss[labels[idx]] += hv_matrix[idx]

    return class_hvss

In [28]:
# with hd.utils.timing("Single pass"):
class_hvss = single_pass(hv_matrix, y_train, N, n_classes)
#     class_hvs.debug_print_values()

[ERROR]	jit.date_type_mutator	
<class '_ast.Module'> : 
  <class '_ast.For'> :  <RPT: N>
    <class '_ast.Assign'> :  [__ARG__class_hvss : __ARG__class_hvss + __ARG__hv_matrix]

{'idx': <class 'int'>, 'N': <class 'int'>, 'class_hvss': 'hypermat_type', 'labels': 'np_int_array_type', 'hv_matrix': 'hypermat_type', '__ARG__hv_matrix': 'float*', '__ARG__labels': 'int*', '__ARG__labels__STRIDE__': 'const int', '__ARG__class_hvss': 'float*', 'hv_matrix_idx_ead026f6': <class 'float'>, 'labels_idx_b9c77ce4': <class 'int'>, 'class_hvss_idx_labels_1b9b16ff': <class 'float'>}


In [29]:
def validate(labels, pred_labels):
    n_correct = (pred_labels == labels).sum()
    n_labels = len(labels)
    print(n_correct, n_labels, n_correct / float(n_labels) * 100)


In [30]:
@hd.run
def retrain(class_hvss, hv_matrix, labels, N, n_classes):
    search_results = hd.search(class_hvss, hv_matrix)

    for idx in range(N):
        if search_results[idx] != labels[idx]:
            class_hvss[labels[idx]] += hv_matrix[idx]
            class_hvss[search_results[idx]] -= hv_matrix[idx]

    return class_hvss


In [31]:
RETRAIN_ITERATIONS = 100
SHOW_STEP_RESULT = True
for it in range(RETRAIN_ITERATIONS):
    with hd.utils.timing("Retrain itereation: %d" % it):
        class_hvss = retrain(class_hvss, hv_matrix, y_train, N, n_classes)

    if SHOW_STEP_RESULT and y_test is not None:
        validate(y_test, hd.search(class_hvss, hv_matrix_test).to_numpy())

[ERROR]	jit.date_type_mutator	
<class '_ast.Module'> : 
  <class '_ast.For'> :  <RPT: N>
    <class '_ast.Assign'> :  [hv_matrix_idx_ead026f6 : __ARG__hv_matrix]
    <class '_ast.If'> : 
      then : 
        <class '_ast.Assign'> :  [__ARG__class_hvss : __ARG__class_hvss + hv_matrix_idx_ead026f6]
        <class '_ast.Assign'> :  [__ARG__class_hvss : __ARG__class_hvss + hv_matrix_idx_ead026f6]
      else : 

{'idx': <class 'int'>, 'N': <class 'int'>, 'search_results': 'np_int_array_type', 'labels': 'np_int_array_type', 'class_hvss': 'hypermat_type', 'hv_matrix': 'hypermat_type', '__ARG__hv_matrix': 'float*', '__ARG__labels': 'int*', '__ARG__labels__STRIDE__': 'const int', '__ARG__search_results': 'int*', '__ARG__search_results__STRIDE__': 'const int', '__ARG__class_hvss': 'float*', 'search_results_idx_b85d35f6': <class 'int'>, 'labels_idx_b9c77ce4': <class 'int'>, 'hv_matrix_idx_ead026f6': <class 'float'>, 'class_hvss_idx_labels_1b9b16ff': <class 'float'>, 'class_hvss_idx_search_result

In [32]:
validate(y_train, hd.search(class_hvss, hv_matrix).to_numpy())

48 135 35.55555555555556


In [33]:
validate(y_train, hd.search(class_hvs, hv_matrix).to_numpy())

48 135 35.55555555555556


In [32]:
with hd.utils.timing("Single pass"):
    hd.search(class_hvss, hv_matrix_test).to_numpy()

Single pass	0.018413782119750977


In [34]:
validate(y_test, hd.search(class_hvss, hv_matrix_test).to_numpy())

3 15 20.0
